In [1]:
import dgl 
from dgl.nn.pytorch.conv import SAGEConv
import numpy as np 
import json 
import torch

import graph_training as g_train

/ext3/conda/zillow_MMKG/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
####################################
## Sample with Correct Formatting ##
####################################

dataset = dgl.data.CSVDataset('./graph_csv')
g = dataset[0]

embedding_length = g.ndata['feat'].size()[1]

conv = SAGEConv(embedding_length, embedding_length, 'mean')
res = conv(g, g.ndata['feat'])
res

Done loading data from cached files.


tensor([[-0.4729, -0.4447, -0.2980,  ...,  0.3795,  0.3539, -0.9945],
        [-0.3101, -0.5475, -0.4219,  ...,  0.4979, -0.1411, -0.8216],
        [-0.3417, -0.2552, -0.1960,  ...,  0.5045,  0.1464, -1.2719],
        ...,
        [-0.2411, -0.5378, -0.1492,  ..., -0.6615, -0.2417, -0.2994],
        [-0.2453, -0.3503, -0.9449,  ...,  0.0665, -0.3339, -0.0525],
        [ 0.0911, -0.1186, -0.9535,  ...,  0.1819, -0.8704, -0.2218]],
       grad_fn=<AddBackward0>)

In [3]:
##################################
## Train -> Evaluation Pipeline ##
##################################

g_train.run(g, 'gcn')

Training...
Epoch 00000 | Loss 0.7841 | Accuracy 585.6627 
Epoch 00001 | Loss 0.5269 | Accuracy 796.2719 
Epoch 00002 | Loss 0.3362 | Accuracy 1035.8636 


/scratch/aks9136/NYU-Zillow-Capstone-2022-Team-A/graph/graph_training.py:53: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  loss = loss_fcn(logits[train_mask], g.ndata['feat'][train_mask])
/ext3/conda/zillow_MMKG/lib/python3.8/site-packages/torch/autograd/__init__.py:191: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/scratch/aks9136/NYU-Zillow-Capstone-2022-Team-A/graph/graph_training.py:36: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  preds = preds[mask]
/sc

Epoch 00003 | Loss 0.2021 | Accuracy 1266.1562 
Epoch 00004 | Loss 0.1041 | Accuracy 1474.4838 
Epoch 00005 | Loss 0.0320 | Accuracy 1657.6792 
Epoch 00006 | Loss -0.0268 | Accuracy 1816.0425 
Epoch 00007 | Loss -0.0732 | Accuracy 1951.6732 
Epoch 00008 | Loss -0.1140 | Accuracy 2067.0828 
Epoch 00009 | Loss -0.1494 | Accuracy 2164.9224 
Epoch 00010 | Loss -0.1808 | Accuracy 2247.7510 
Epoch 00011 | Loss -0.2153 | Accuracy 2317.6465 
Epoch 00012 | Loss -0.2405 | Accuracy 2376.7378 
Epoch 00013 | Loss -0.2658 | Accuracy 2426.8801 
Epoch 00014 | Loss -0.2889 | Accuracy 2469.7158 
Epoch 00015 | Loss -0.3123 | Accuracy 2506.6279 
Epoch 00016 | Loss -0.3352 | Accuracy 2538.7554 
Epoch 00017 | Loss -0.3537 | Accuracy 2567.2026 
Epoch 00018 | Loss -0.3684 | Accuracy 2592.9312 
Epoch 00019 | Loss -0.3857 | Accuracy 2616.7214 
Epoch 00020 | Loss -0.4049 | Accuracy 2639.0396 
Epoch 00021 | Loss -0.4211 | Accuracy 2660.3613 
Epoch 00022 | Loss -0.4366 | Accuracy 2681.1191 
Epoch 00023 | Loss -0.4